<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [43]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [44]:
import numpy as np
import time
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")


**SAWMILL CLASS**

In [46]:
#SawMill Object
m = GEKKO(remote=True)

class Mill:
  
  def __init__(self, Id, Wood, Prod, Dis, Dem=0, Radio = 20):
    self.Id1 = Id
    self.Wood_type =Wood
    self.Prod_type = Prod
    self.Dist = Dis
    self.Demand = Dem
    self.Radio = Radio
  

  def Give_Id(self):
    a = 2
    return a 


  def Harvest_Close(self, Available, Dem):
    
    Num = len(self.Dist)
  
    H = m.Array(m.Var,(Num),value = 0)

    for i in range(Num):
      H[i].lower = 0
      H[i].upper = int(Available[i])
      
    prod0 = m.Intermediate(m.sum([self.Dist.loc[i,('DIST')]*H[i] for i in range(num)]))
    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=True)
    a = self.Dist
    flat = [item for l in H for item in l]
    a['HARVESTED'] = flat 
    return(a)
    

In [73]:
class Inventory:
  def __init__(self, Av, Dist):
    self.Available = []
    self.Available.append(Av) #Dataframe that contains Available wood Inventory at diferent time t
    self.Dist = Dist #Matrix Distance from any cell to another cell
    self.Mills = [] #Dataframe that contains Id, typewood, type_product, and demand of mills
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested


  def Add_Mill2(self, Id, Wood, Prod, Dem=0):

    a =self.Cell_Less(Id,0,20)
    D2 = a.loc[:, ('DEST','DIST')]
    mil= Mill(id,Wood,Prod,D2,Dem)
    self.Mills.append(mil)
  

  def Harvest_Mill(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    Id1 =self.Mills.loc[j,('Unique_ID')]
    a =self.Cell_Less(Id1,0,20)
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]
    Num = len(self.Dist_Mill)
    
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t, 20)
    print('Available',self.Available_Mill)

    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))
    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=True)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['HARVESTED'] = flat 
    ans = a
    

    #ans = self.Mills[j].Harvest_Close(self.Available[t].SumOfTotal,self.Mills[j].Demand)
    
    for j in range(len(ans)):                                                                         
      if not(self.Available[0][self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[0].loc[a2,("SumOfTotal")] = self.Available[0].loc[a2,("SumOfTotal")] - ans.loc[j,('HARVESTED')]
      
  
  def Harvest_All_Mills(self):
    for k in range(len(self.Mills)):
      Hardwood_Sawlog.Harvest_Mill(0,k)
    
  
  def Cell_Less(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of cells with Distance and Availability
    #print('Id:', Id_Mill)
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    #print('Ans:',Ans)
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))

    Ans["AVAIL"] = Disp 
    return(Ans)

  def Disp_Radio(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))

    return(Disp)
  

  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    #print("Numero de celdas ",len(Ans))
    return(len(Ans))
    
  




**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [74]:
# Main Program
start_time = time.time()
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")

#df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
#                   columns=['Mill_Id', 'wood', 'type','Demand'])
#print("tipo",type(Hardwood_Sawlog_Data))

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
#print(a.describe())
v1 = Hardwood_Sawlog.Available[0]["Unique_ID"]==54549
#Hardwood_Sawlog.Available[0].loc[v1,("Unique_ID","SumOfTotal")]
#print('C2:',c2)
v2 =int(Hardwood_Sawlog.Available[0].loc[v1,("SumOfTotal")])
print('Before harvesting',v2)

#print('Dist',Hardwood_Sawlog.Dist)

ma = Dist_Data['ORIG'].unique()
print('Ma', ma[0])

#Mills_Data.loc[0, ('Unique_ID')]

for k in range(1):
  Mills_Data.loc[k,('Unique_ID')] = ma[k]
  Mills_Data.loc[k,('Demand')] = 10000000
  #print('Id2: ',Mills_Data.loc[0,('Unique_ID')])
  #Hardwood_Sawlog.Mills.append(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][k]))
  #mi= Mill(Mills_Data.loc[k, ('Unique_ID')],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Hardwood_Sawlog.Dist.loc[:, ('DIST')],Mills_Data.loc[k,('Demand')])
  #Hardwood_Sawlog.Add_Mill(mi)
  #Hardwood_Sawlog.Add_Mill2(Mills_Data.loc[k, ('Unique_ID')],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')])
  #Hardwood_Sawlog.Add_Mill2(ma[k],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')])
#  saw=[ma[k],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')]]
  #Hardwood_Sawlog.Mills.append(saw)
 # print(Hardwood_Sawlog.Mills.loc[k,('Id')])
  #Hardwood_Sawlog.Add_Mill2(ma[k],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],10000000)
  #print("Cosechando Aserradero ", k)
  #Hardwood_Sawlog.Harvest_Mill(0,k)

Hardwood_Sawlog.Mills=Mills_Data
Hardwood_Sawlog.Harvest_Mill(0,0)
#print('Saw:', Hardwood_Sawlog.Mills)
  
print('AFTER HARVEST',Hardwood_Sawlog.Available[0].loc[v1,("SumOfTotal")])

print ("My program took", time.time() - start_time, "to run")

Before harvesting 2287844
Ma 54549


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Available [2287844, 2268199, 2280210, 2287724, 2294929, 2196091, 2294260, 2155774, 0, 2298647, 2295427, 2279530, 2219389, 0, 2293918, 2287595, 0, 0, 2300877, 0, 2173385, 2362789, 2314771, 2148396, 0, 0, 2278208, 2270021, 2208712, 0, 0, 0, 0, 2287189, 0, 0, 0, 2392038, 2378180, 0, 2263843, 0, 2395996, 2261314, 0, 0, 0, 2267843, 0, 2262769, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2252419, 2249340, 0, 2393025, 2258948, 2398536, 2329011, 0, 2384037, 0, 0, 0, 0, 0, 2276705, 0, 2286946, 0, 2256814, 0, 0, 2256799, 0, 2265424, 0, 0, 0, 0, 0, 0, 2401206, 2240541, 2246756, 0, 2373163, 2237643, 2393533, 0, 0, 2382677, 0, 0, 0, 2335616, 0, 0, 0, 0, 0, 2243754, 2403003, 2253545, 0, 2251001, 0, 0, 2392513, 2262503, 0, 0, 0, 0, 0, 2407123, 2229357, 0, 0, 2370774, 0, 0, 0, 2225729, 0, 0, 2234264, 2380491, 0, 2288386, 0, 2358333, 0, 0, 0, 0, 0, 0, 0, 0, 2250200, 0, 2249468, 0, 0, 2390305, 0, 0, 2221723, 0, 2402974, 0, 2240808, 0, 0, 2246890, 0, 0, 0, 0, 2184440, 0, 2220668, 0]
AFTER HARVEST 5096    0.7
Name: SumOfT